# 3. Classification

The most common supervised learning tasks are **regression** and **classification**. We already worked through a ML project involving regression (housing prices), so it's time to give classification some of the attention it deserves.  

#### MNIST

In this Chapter, we will use the classic MNIST dataset, probably the most common dataset for image classification. 

Indeed, we can download MNIST directly from scikit-learn:  